<a href="https://colab.research.google.com/github/pulavartivinay/111801034/blob/main/Pulavarti_VinayKumar_111801034_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#import and necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, LSTM, Embedding, GRU, SimpleRNN

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Data set-1**

In [2]:
!gdown --id 1FbPIlE56GpQrzKCHfXtg6Rd7PVkJe-Qa

Downloading...
From: https://drive.google.com/uc?id=1FbPIlE56GpQrzKCHfXtg6Rd7PVkJe-Qa
To: /content/dataset_1.tsv
100% 85.3k/85.3k [00:00<00:00, 63.0MB/s]


In [30]:
dataSet_Frame1 = pd.read_csv("dataset_1.tsv",  sep="[\t\n]", engine='python', header=None)
dataSet_Frame1.columns = ["reviews", "labels"]

dataSet_X1 = dataSet_Frame1["reviews"].to_numpy()
dataSet_y1 = dataSet_Frame1["labels"].to_numpy()
print(dataSet_Frame1.head())

                                             reviews  labels
0  A very, very, very slow-moving, aimless movie ...       0
1  Not sure who was more lost - the flat characte...       0
2  Attempting artiness with black & white and cle...       0
3       Very little music or anything to speak of.         0
4  The best scene in the movie was when Gerardo i...       1


**Data Preprocessing for Data set-1**

In [31]:
stopWords = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemm = WordNetLemmatizer()
punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''

filtered_dataSet_X1 = []
for i in range(len(dataSet_X1)):
  dataSet_X1[i] = dataSet_X1[i].lower()
  dataSet_X1[i] = dataSet_X1[i].replace('[^\w\s]','')
  for ch in dataSet_X1[i]:
        if ch in punctuations:
          dataSet_X1[i] = dataSet_X1[i].replace(ch, '')
  words = word_tokenize(dataSet_X1[i])
  filtered_words = []
  for w in words:
    if w not in stopWords:
      w = stemmer.stem(w)
      w = lemm.lemmatize(w)
      filtered_words.append(w)

  filtered_dataSet_X1.append(filtered_words)


print(filtered_dataSet_X1)



[['slowmov', 'aimless', 'movi', 'distress', 'drift', 'young', 'man'], ['sure', 'lost', 'flat', 'charact', 'audienc', 'nearli', 'half', 'walk'], ['attempt', 'arti', 'black', 'white', 'clever', 'camera', 'angl', 'movi', 'disappoint', 'becam', 'even', 'ridicul', 'act', 'poor', 'plot', 'line', 'almost', 'nonexist'], ['littl', 'music', 'anyth', 'speak'], ['best', 'scene', 'movi', 'gerardo', 'tri', 'find', 'song', 'keep', 'run', 'head'], ['rest', 'movi', 'lack', 'art', 'charm', 'mean', 'empti', 'work', 'guess', 'empti'], ['wast', 'two', 'hour'], ['saw', 'movi', 'today', 'thought', 'good', 'effort', 'good', 'messag', 'kid'], ['bit', 'predict'], ['love', 'cast', 'jimmi', 'buffet', 'scienc', 'teacher'], ['babi', 'owl', 'ador'], ['movi', 'show', 'lot', 'florida', 'best', 'made', 'look', 'appeal'], ['song', 'best', 'muppet', 'hilari'], ['cool'], ['right', 'case', 'movi', 'deliv', 'everyth', 'almost', 'right', 'face'], ['averag', 'act', 'main', 'person', 'low', 'budget', 'clearli', 'see'], ['revie

In [32]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n', lower=True)
tk.fit_on_texts(filtered_dataSet_X1)
X_seq = tk.texts_to_sequences(filtered_dataSet_X1)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')
print(X_pad)
X_pad.shape

[[1008 1009    1 ...    0    0    0]
 [ 339  340  637 ...    0    0    0]
 [ 149 1012  113 ...    0    0    0]
 ...
 [ 185  378    0 ...    0    0    0]
 [ 448    4    0 ...    0    0    0]
 [ 437    3  211 ...    0    0    0]]


(1000, 100)

**splitting the data set-1 into test and train**

In [33]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_pad, dataSet_y1, test_size=0.2, random_state=14, shuffle=True)

In [34]:
len(tk.word_counts)

2529

**RNN Model**

In [61]:
model = Sequential()
model.add(Embedding(2530, 100))
model.add(SimpleRNN(192, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(SimpleRNN(128, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 100)         253000    
_________________________________________________________________
simple_rnn_16 (SimpleRNN)    (None, None, 192)         56256     
_________________________________________________________________
simple_rnn_17 (SimpleRNN)    (None, 128)               41088     
_________________________________________________________________
dense_27 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_28 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 33        
Total params: 360,713
Trainable params: 360,713
Non-trainable params: 0
_______________________________________________

In [62]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [63]:
model.fit(X1_train, y1_train, validation_data=(X1_test, y1_test),epochs = 20, verbose=1, batch_size=16)

Epoch 1/20
50/50 [==============================] - 11s 177ms/step - loss: 0.7039 - accuracy: 0.5030 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 2/20
50/50 [==============================] - 8s 166ms/step - loss: 0.7065 - accuracy: 0.4751 - val_loss: 0.6917 - val_accuracy: 0.5250
Epoch 3/20
50/50 [==============================] - 8s 163ms/step - loss: 0.7123 - accuracy: 0.4620 - val_loss: 0.6921 - val_accuracy: 0.5250
Epoch 4/20
50/50 [==============================] - 8s 160ms/step - loss: 0.6915 - accuracy: 0.5243 - val_loss: 0.6947 - val_accuracy: 0.4750
Epoch 5/20
50/50 [==============================] - 8s 163ms/step - loss: 0.7037 - accuracy: 0.4699 - val_loss: 0.6945 - val_accuracy: 0.4750
Epoch 6/20
50/50 [==============================] - 8s 164ms/step - loss: 0.7050 - accuracy: 0.4686 - val_loss: 0.7007 - val_accuracy: 0.4750
Epoch 7/20
50/50 [==============================] - 8s 159ms/step - loss: 0.6942 - accuracy: 0.5120 - val_loss: 0.7031 - val_accuracy: 0.4750
Epoch

**LSTM Model**

In [64]:
model = Sequential()
model.add(Embedding(2530, 100))
model.add(LSTM(192, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 100)         253000    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 192)         225024    
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               164352    
_________________________________________________________________
dense_30 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_31 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 33        
Total params: 652,745
Trainable params: 652,745
Non-trainable params: 0
_______________________________________________

In [65]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [66]:
model.fit(X1_train, y1_train, validation_data=(X1_test, y1_test),epochs = 20, verbose=1)

Epoch 1/20
25/25 [==============================] - 14s 422ms/step - loss: 0.7083 - accuracy: 0.4441 - val_loss: 0.6934 - val_accuracy: 0.4900
Epoch 2/20
25/25 [==============================] - 10s 389ms/step - loss: 0.6962 - accuracy: 0.4716 - val_loss: 0.6936 - val_accuracy: 0.4750
Epoch 3/20
25/25 [==============================] - 10s 392ms/step - loss: 0.6926 - accuracy: 0.5040 - val_loss: 0.6932 - val_accuracy: 0.4750
Epoch 4/20
25/25 [==============================] - 10s 398ms/step - loss: 0.6955 - accuracy: 0.5329 - val_loss: 0.6937 - val_accuracy: 0.4750
Epoch 5/20
25/25 [==============================] - 10s 395ms/step - loss: 0.6933 - accuracy: 0.4967 - val_loss: 0.6934 - val_accuracy: 0.4750
Epoch 6/20
25/25 [==============================] - 10s 399ms/step - loss: 0.6931 - accuracy: 0.5160 - val_loss: 0.6937 - val_accuracy: 0.4750
Epoch 7/20
25/25 [==============================] - 10s 392ms/step - loss: 0.6935 - accuracy: 0.4856 - val_loss: 0.6936 - val_accuracy: 0.4750

**GRU MODEL**

In [67]:
model = Sequential()
model.add(Embedding(2530, 100))
model.add(LSTM(192, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 100)         253000    
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 192)         225024    
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               164352    
_________________________________________________________________
dense_33 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_34 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 33        
Total params: 652,745
Trainable params: 652,745
Non-trainable params: 0
_______________________________________________

In [68]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [69]:
model.fit(X1_train, y1_train, validation_data=(X1_test, y1_test),epochs = 20, verbose=1)

Epoch 1/20
25/25 [==============================] - 15s 427ms/step - loss: 0.6934 - accuracy: 0.4830 - val_loss: 0.6931 - val_accuracy: 0.5250
Epoch 2/20
25/25 [==============================] - 10s 402ms/step - loss: 0.6932 - accuracy: 0.5195 - val_loss: 0.6932 - val_accuracy: 0.4750
Epoch 3/20
25/25 [==============================] - 10s 396ms/step - loss: 0.6933 - accuracy: 0.4942 - val_loss: 0.6932 - val_accuracy: 0.4750
Epoch 4/20
25/25 [==============================] - 10s 402ms/step - loss: 0.6931 - accuracy: 0.5291 - val_loss: 0.6937 - val_accuracy: 0.4750
Epoch 5/20
25/25 [==============================] - 10s 401ms/step - loss: 0.6931 - accuracy: 0.5118 - val_loss: 0.6935 - val_accuracy: 0.4750
Epoch 6/20
25/25 [==============================] - 10s 401ms/step - loss: 0.6932 - accuracy: 0.4967 - val_loss: 0.6935 - val_accuracy: 0.4750
Epoch 7/20
25/25 [==============================] - 10s 403ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6935 - val_accuracy: 0.4750

# **Comparison Between the Models for Data set-1**


| $Metrics/Models$  | $SimpleRNN$ | $LSTM$    | $GRU$     |
|:-----------------:|:-----------:|:---------:|:---------:|
|   Accuracy        |   47.50     |   47.50   |   47.50   |

**Data set-2**

In [73]:
!unzip /content/dataset_2-20210426T102044Z-001.zip

Archive:  /content/dataset_2-20210426T102044Z-001.zip
  inflating: dataset_2/rt-polarity.neg  
  inflating: dataset_2/rt-polarity.pos  


In [74]:
dataSet_Frame2_positive = pd.read_table('/content/dataset_2/rt-polarity.pos',header=0,names=['reviews'])
dataSet_Frame2_positive['labels'] = np.ones(dataSet_Frame2_positive.shape[0],dtype=np.int32)

In [75]:
dataSet_Frame2_positive.head()

,reviews,labels
0,"the gorgeously elaborate continuation of "" the...",1
1,effective but too-tepid biopic,1
2,if you sometimes like to go to the movies to h...,1
3,"emerges as something rare , an issue movie tha...",1
4,the film provides some great insight into the ...,1


In [76]:
dataSet_Frame2_negative = pd.read_table('/content/dataset_2/rt-polarity.neg',header=0,names=['reviews'])
dataSet_Frame2_negative['labels'] = np.zeros(dataSet_Frame2_negative.shape[0],dtype=np.int32)

In [77]:
dataSet_Frame2_negative.head()

,reviews,labels
0,"it's so laddish and juvenile , only teenage bo...",0
1,exploitative and largely devoid of the depth o...,0
2,[garbus] discards the potential for pathologic...,0
3,a visually flashy but narratively opaque and e...,0
4,"the story is also as unoriginal as they come ,...",0


**Combining the positive and negative reviews in one dataset**

In [78]:
dataSet_Frame2 = pd.concat([dataSet_Frame2_positive, dataSet_Frame2_negative])

In [79]:
dataSet_Frame2.head()

,reviews,labels
0,"the gorgeously elaborate continuation of "" the...",1
1,effective but too-tepid biopic,1
2,if you sometimes like to go to the movies to h...,1
3,"emerges as something rare , an issue movie tha...",1
4,the film provides some great insight into the ...,1


In [80]:
dataSet_X2 = dataSet_Frame2["reviews"].to_numpy()
dataSet_y2 = dataSet_Frame2["labels"].to_numpy()
print(dataSet_Frame2.head())

                                             reviews  labels
0  the gorgeously elaborate continuation of " the...       1
1                     effective but too-tepid biopic       1
2  if you sometimes like to go to the movies to h...       1
3  emerges as something rare , an issue movie tha...       1
4  the film provides some great insight into the ...       1


**Data Preprocessing for Data set-2** 

In [81]:
stopWords = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemm = WordNetLemmatizer()
punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''

filtered_dataSet_X2 = []
for i in range(len(dataSet_X2)):
  dataSet_X2[i] = dataSet_X2[i].lower()
  dataSet_X2[i] = dataSet_X2[i].replace('[^\w\s]','')
  for ch in dataSet_X2[i]:
        if ch in punctuations:
          dataSet_X2[i] = dataSet_X2[i].replace(ch, '')
  words = word_tokenize(dataSet_X2[i])
  filtered_words = []
  for w in words:
    if w not in stopWords:
      w = stemmer.stem(w)
      w = lemm.lemmatize(w)
      filtered_words.append(w)

  filtered_dataSet_X2.append(filtered_words)


print(filtered_dataSet_X2)



[['gorgeous', 'elabor', 'continu', 'lord', 'ring', 'trilog', 'huge', 'column', 'word', 'adequ', 'describ', 'cowriterdirector', 'peter', 'jackson', 'expand', 'vision', 'j', 'r', 'r', 'tolkien', 'middleearth'], ['effect', 'tootepid', 'biopic'], ['sometim', 'like', 'go', 'movi', 'fun', 'wasabi', 'good', 'place', 'start'], ['emerg', 'someth', 'rare', 'issu', 'movi', 'that', 'honest', 'keenli', 'observ', 'doesnt', 'feel', 'like', 'one'], ['film', 'provid', 'great', 'insight', 'neurot', 'mindset', 'comic', 'even', 'reach', 'absolut', 'top', 'game'], ['offer', 'rare', 'combin', 'entertain', 'educ'], ['perhap', 'pictur', 'ever', 'made', 'liter', 'show', 'road', 'hell', 'pave', 'good', 'intent'], ['steer', 'turn', 'snappi', 'screenplay', 'curl', 'edg', 'clever', 'want', 'hate', 'somehow', 'pull'], ['take', 'care', 'cat', 'offer', 'refreshingli', 'differ', 'slice', 'asian', 'cinema'], ['film', 'well', 'worth', 'see', 'talk', 'sing', 'head'], ['realli', 'surpris', 'wisegirl', 'lowkey', 'qualiti',

In [82]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n', lower=True)
tk.fit_on_texts(filtered_dataSet_X2)
X_seq = tk.texts_to_sequences(filtered_dataSet_X2)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')
print(X_pad)
X_pad.shape

[[2588 1786  819 ...    0    0    0]
 [ 119 7795 1787 ...    0    0    0]
 [ 199    3   44 ...    0    0    0]
 ...
 [ 453 4382 2638 ...    0    0    0]
 [  30   20    3 ...    0    0    0]
 [2316 1391  980 ...    0    0    0]]


(10660, 100)

**splitting the data set-2 into test and train**

In [83]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X_pad, dataSet_y2, test_size=0.2, random_state=14, shuffle=True)

**RNN Model**

In [84]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(SimpleRNN(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(SimpleRNN(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
simple_rnn_18 (SimpleRNN)    (None, None, 256)         91392     
_________________________________________________________________
simple_rnn_19 (SimpleRNN)    (None, 256)               131328    
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 257       
Total params: 2,222,977
Trainable params: 2,222,977
Non-trainable params: 0
_________________________________________________________________


In [85]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [87]:
model.fit(X2_train, y2_train, validation_data=(X2_test, y2_test),epochs = 5, verbose=1, batch_size=64)

Epoch 1/5
134/134 [==============================] - 46s 346ms/step - loss: 0.7180 - accuracy: 0.4986 - val_loss: 0.7174 - val_accuracy: 0.4920
Epoch 2/5
134/134 [==============================] - 46s 344ms/step - loss: 0.7106 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.5080
Epoch 3/5
134/134 [==============================] - 46s 343ms/step - loss: 0.7092 - accuracy: 0.4987 - val_loss: 0.7029 - val_accuracy: 0.5080
Epoch 4/5
134/134 [==============================] - 46s 341ms/step - loss: 0.7062 - accuracy: 0.4974 - val_loss: 0.6977 - val_accuracy: 0.5080
Epoch 5/5
134/134 [==============================] - 46s 344ms/step - loss: 0.7044 - accuracy: 0.5035 - val_loss: 0.6937 - val_accuracy: 0.5019


**LSTM Model**

In [89]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 256)         365568    
_________________________________________________________________
lstm_11 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 257       
Total params: 2,891,137
Trainable params: 2,891,137
Non-trainable params: 0
_________________________________________________________________


In [90]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [91]:
model.fit(X2_train, y2_train, validation_data=(X2_test, y2_test),epochs = 5, verbose=1, batch_size=64)

Epoch 1/5
134/134 [==============================] - 145s 1s/step - loss: 0.6934 - accuracy: 0.4930 - val_loss: 0.6936 - val_accuracy: 0.4920
Epoch 2/5
134/134 [==============================] - 136s 1s/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6930 - val_accuracy: 0.5080
Epoch 3/5
134/134 [==============================] - 135s 1s/step - loss: 0.6934 - accuracy: 0.4948 - val_loss: 0.6932 - val_accuracy: 0.4920
Epoch 4/5
134/134 [==============================] - 134s 998ms/step - loss: 0.6932 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.4920
Epoch 5/5
  6/134 [>.............................] - ETA: 2:11 - loss: 0.6933 - accuracy: 0.4652

KeyboardInterrupt: ignored

**GRU Model**

In [92]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
gru (GRU)                    (None, None, 256)         274944    
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 257       
Total params: 2,669,953
Trainable params: 2,669,953
Non-trainable params: 0
_________________________________________________________________


In [93]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [95]:
model.fit(X2_train, y2_train, validation_data=(X2_test, y2_test),epochs = 2, verbose=1, batch_size=64)

Epoch 1/2
134/134 [==============================] - 122s 914ms/step - loss: 0.6937 - accuracy: 0.4959 - val_loss: 0.6935 - val_accuracy: 0.4920
Epoch 2/2
134/134 [==============================] - 121s 907ms/step - loss: 0.6932 - accuracy: 0.5055 - val_loss: 0.6930 - val_accuracy: 0.5080


# **Comparison Between the Models for Data set-2**


| $Metrics/Models$  | $SimpleRNN$ | $LSTM$    | $GRU$     |
|:-----------------:|:-----------:|:---------:|:---------:|
|   Accuracy        |   50.19     |   49.20   |   50.80   |

**Combining Two data sets**

In [96]:
dataSet_Frame_Combined = pd.concat([dataSet_Frame1 , dataSet_Frame2])

In [97]:
dataSet_X = dataSet_Frame_Combined["reviews"].to_numpy()
dataSet_y = dataSet_Frame_Combined["labels"].to_numpy()

**Data Preprocessing for Combined Data set** 

In [98]:
stopWords = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemm = WordNetLemmatizer()
punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''

filtered_dataSet_X = []
for i in range(len(dataSet_X)):
  dataSet_X[i] = dataSet_X[i].lower()
  dataSet_X[i] = dataSet_X[i].replace('[^\w\s]','')
  for ch in dataSet_X[i]:
        if ch in punctuations:
          dataSet_X[i] = dataSet_X[i].replace(ch, '')
  words = word_tokenize(dataSet_X[i])
  filtered_words = []
  for w in words:
    if w not in stopWords:
      w = stemmer.stem(w)
      w = lemm.lemmatize(w)
      filtered_words.append(w)

  filtered_dataSet_X.append(filtered_words)


print(filtered_dataSet_X)



[['slowmov', 'aimless', 'movi', 'distress', 'drift', 'young', 'man'], ['sure', 'lost', 'flat', 'charact', 'audienc', 'nearli', 'half', 'walk'], ['attempt', 'arti', 'black', 'white', 'clever', 'camera', 'angl', 'movi', 'disappoint', 'becam', 'even', 'ridicul', 'act', 'poor', 'plot', 'line', 'almost', 'nonexist'], ['littl', 'music', 'anyth', 'speak'], ['best', 'scene', 'movi', 'gerardo', 'tri', 'find', 'song', 'keep', 'run', 'head'], ['rest', 'movi', 'lack', 'art', 'charm', 'mean', 'empti', 'work', 'guess', 'empti'], ['wast', 'two', 'hour'], ['saw', 'movi', 'today', 'thought', 'good', 'effort', 'good', 'messag', 'kid'], ['bit', 'predict'], ['love', 'cast', 'jimmi', 'buffet', 'scienc', 'teacher'], ['babi', 'owl', 'ador'], ['movi', 'show', 'lot', 'florida', 'best', 'made', 'look', 'appeal'], ['song', 'best', 'muppet', 'hilari'], ['cool'], ['right', 'case', 'movi', 'deliv', 'everyth', 'almost', 'right', 'face'], ['averag', 'act', 'main', 'person', 'low', 'budget', 'clearli', 'see'], ['revie

In [99]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n', lower=True)
tk.fit_on_texts(filtered_dataSet_X)
X_seq = tk.texts_to_sequences(filtered_dataSet_X)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')
print(X_pad)
X_pad.shape

[[3914 2442    2 ...    0    0    0]
 [ 268  546  520 ...    0    0    0]
 [ 244 3026  352 ...    0    0    0]
 ...
 [ 458 4545 2773 ...    0    0    0]
 [  28   19    3 ...    0    0    0]
 [2430 1485  993 ...    0    0    0]]


(11660, 100)

**splitting the Combined data set into test and train**

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, dataSet_y, test_size=0.2, random_state=14, shuffle=True)

**RNN Model**

In [101]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(SimpleRNN(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(SimpleRNN(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
simple_rnn_22 (SimpleRNN)    (None, None, 256)         91392     
_________________________________________________________________
simple_rnn_23 (SimpleRNN)    (None, 256)               131328    
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 257       
Total params: 2,222,977
Trainable params: 2,222,977
Non-trainable params: 0
_________________________________________________________________


In [102]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [103]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs = 5, verbose=1, batch_size=64)

Epoch 1/5
146/146 [==============================] - 51s 336ms/step - loss: 0.7307 - accuracy: 0.4936 - val_loss: 0.6976 - val_accuracy: 0.4871
Epoch 2/5
146/146 [==============================] - 49s 333ms/step - loss: 0.7144 - accuracy: 0.4993 - val_loss: 0.6979 - val_accuracy: 0.4833
Epoch 3/5
146/146 [==============================] - 49s 337ms/step - loss: 0.7111 - accuracy: 0.4968 - val_loss: 0.7007 - val_accuracy: 0.4833
Epoch 4/5
146/146 [==============================] - 50s 342ms/step - loss: 0.7042 - accuracy: 0.5027 - val_loss: 0.6935 - val_accuracy: 0.4927
Epoch 5/5
146/146 [==============================] - 48s 332ms/step - loss: 0.7025 - accuracy: 0.5018 - val_loss: 0.7004 - val_accuracy: 0.4828


**LSTM Model**

In [104]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
lstm_12 (LSTM)               (None, None, 256)         365568    
_________________________________________________________________
lstm_13 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 257       
Total params: 2,891,137
Trainable params: 2,891,137
Non-trainable params: 0
_________________________________________________________________


In [105]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [106]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs = 2, verbose=1, batch_size=64)

Epoch 1/2
146/146 [==============================] - 151s 1s/step - loss: 0.6934 - accuracy: 0.5012 - val_loss: 0.6930 - val_accuracy: 0.5167
Epoch 2/2
146/146 [==============================] - 144s 989ms/step - loss: 0.6932 - accuracy: 0.5048 - val_loss: 0.6933 - val_accuracy: 0.4833


**GRU Model**

In [107]:
model = Sequential()
model.add(Embedding(20000, 100))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 100)         2000000   
_________________________________________________________________
gru_2 (GRU)                  (None, None, 256)         274944    
_________________________________________________________________
gru_3 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 257       
Total params: 2,669,953
Trainable params: 2,669,953
Non-trainable params: 0
_________________________________________________________________


In [108]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [109]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs = 1, verbose=1, batch_size=64)

146/146 [==============================] - 138s 914ms/step - loss: 0.6933 - accuracy: 0.5023 - val_loss: 0.6941 - val_accuracy: 0.4833


# **Comparison Between the Models for Combined Data set**


| $Metrics/Models$  | $SimpleRNN$ | $LSTM$    | $GRU$     |
|:-----------------:|:-----------:|:---------:|:---------:|
|   Accuracy        |   48.28     |   48.33   |   48.33   |